In [2]:
import numpy as np
import random
import math
import pandas as pd

In [3]:
# Cargar los datos de Excel
df_distance_km = pd.read_excel('../dataset/df_distance_km.xlsx')
df_distance_min = pd.read_excel('../dataset/df_distance_min.xlsx')
df_historic_order_demand = pd.read_excel('../dataset/df_historic_order_demand.xlsx')
df_location = pd.read_excel('../dataset/df_location.xlsx')
df_orders = pd.read_excel('../dataset/df_orders.xlsx')
df_vehicle = pd.read_excel('../dataset/df_vehicle.xlsx')


In [ ]:
# Ver los primeros registros para asegurarnos que todo se cargó correctamente
print(df_distance_km.head())
print(df_distance_min.head())
print(df_historic_order_demand.head())
print(df_location.head())
print(df_orders.head())
print(df_vehicle.head())

   Cliente_1  Cliente_2  Cliente_3  Cliente_4  Cliente_5  Cliente_6  \
0        0.0     7.5625    15.5365     1.1998     4.7145     1.7407   

   Cliente_7  Cliente_8  Cliente_9  Cliente_10  ...  Cliente_12  Cliente_13  \
0     7.9408    17.1947     4.2933      3.2659  ...      6.0225       5.447   

   Cliente_14  Cliente_15  Cliente_16  Cliente_17  Cliente_18  Cliente_19  \
0      2.2133     11.1505      1.5775     10.8288      9.1456     20.4871   

   Cliente_20  Almacén  
0     22.1445   3.6114  

[1 rows x 21 columns]
   Cliente_1  Cliente_2  Cliente_3  Cliente_4  Cliente_5  Cliente_6  \
0        0.0  15.731667  19.651667   2.543333       7.55       3.76   

   Cliente_7  Cliente_8  Cliente_9  Cliente_10  ...  Cliente_12  Cliente_13  \
0  15.838333  22.388333   8.563333       7.185  ...   10.028333   12.336667   

   Cliente_14  Cliente_15  Cliente_16  Cliente_17  Cliente_18  Cliente_19  \
0    5.098333      17.525    2.943333   15.548333      15.105      23.855   

   Cliente_20